#  Фреймворк Scrapy. Скачивание файлов и фото

C помощью модуля `requests` можно скачивать файлы. Помимо скрапи есть более лёгкие методы скачать файлы с помощью которых можно достаточно эффективно решить данную задачу. Импортируем модуль. Он умеет делать гет запросы. Процесс получения ответа от сервера практически идентичен скачиванию файла с какого либо ресурса. В случае, когда мы качаем файл мы скачиваем данные в этот файл. Когда мы скачиваем страницу - мы скачиваем html файл, просто он раскрывается в нашем браузере и мы получаем содержимое страницы. 
Делаем мы запрос на урл или на ссылку которая приведёт к скачиванию файла схожие процессы, технически никакой разницы нет. 

In [1]:
import requests

Идём на яндекс картинки, выбираем понравившуюся, ПКМ - копировать адрес ссылки. https://kartinkin.net/uploads/posts/2022-12/1671751945_kartinkin-net-p-kot-kosmonavt-kartinki-krasivo-10.jpg 
откроем в отдельном окне. 

У открытой странички есть ссылка на неё. 

In [7]:
url = 'https://kartinkin.net/uploads/posts/2022-12/1671751945_kartinkin-net-p-kot-kosmonavt-kartinki-krasivo-10.jpg'

Выполняем гет запрос через модуль `request` 

In [8]:
response = requests.get(url)

Что дальше делать с этим респонсом. По факту в этом респонсе будет находиться тот самый скаченный файл, в нашем случае это картинка. И нам это нужно превратит в некоторое физическое проявление. 

Воспользуемся стандартной процедурой сохранения данных в файл. Открываем файл на запись причём в бинарном режиме. Мы будем писать сюда не текстовые данные а последовательность нулей и единиц, которые будут хрониться в нашем респонсе. Т.к открываем в бинарном режиме, кодировка никакая не нужна. 

Внутри функции `open` мы возьмём наш файл, вызовим метод `write()` и в него добавим содержимое респонса, но не текст, а контент? который содержит бинарное содержание контента и выполнение гет запроса на указанную ссылку. 

In [9]:
with open('image.jpg', 'wb') as picture:
    picture.write(response.content)

При наличии прямой ссылки на файл скачиваем его без проблем. Но у данного продемонстрированного подхода есть 2 недостатка:

- наш файл целиком и полностью грузится в оперативку - response. И пока он дойдёт от `response = requests.get(url)` до `  picture.write(response.content)` он провисает в оперативке. И если файлов много скачивается в потоках оперативка может забиться. 

- мы делаем фиксированное имя файла. 'image.jpg'. И мы должны заботиться о том, чтобы это имя файла постоянно как то менялось, иначе будем затирать старое новым. 

Для решения проблем можем воспользоваться встроенными средствами. Напрмер скачивание файла по частям. Для имени распарсить ссылку и брать имя картинки оттуда. Можем воспользоваться модулем `wget`. Но изначально его нет, его надо установить. 

In [12]:
! pip install wget


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9682 sha256=f6d2658b14e138a70f0ec07d3fba5cdb8c4b0adad1fc9ea3f1ecfd6f93696ca0
  Stored in directory: c:\users\777\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [13]:
import wget

Этот модуль предоставляет различные возможности. Воспользуемся методом download(). Он сохранит картинку с тем же именем, которое она имеет в ссылке. Также метод автоматически делит файлы на несколько частей, то есть для большого файла используется собственный алгоритм и он не закидывает целиком респонс в оперативку. Файл полюбому движится с сервера в оперативку, с оперативки в жёсткий диск. 

In [14]:
wget.download(url)

'1671751945_kartinkin-net-p-kot-kosmonavt-kartinki-krasivo-10.jpg'

### Для скрапи. создали проект. Создался каркас из файлов. 

В файле `settings.py`:
`BOT_NAME = 'avitoparser'` - название нашего проекта.

`ROBOTSTXT_OBEY = False` - роботы выключены. 

`ITEM_PIPELINES =` - пайплайн один включен.

В самом пауке `spiders/avito.py` классический набор

```
class AvitoSpider(scrapy.Spider):
    name = 'avito'
    allowed_domains = ['avito.ru']
    start_urls = [f'https://www.avito.ru']
```

в пайплайнах `pipelines.py` тоже пустота:
```
class AvitoparserPipeline:
    def process_item(self, item, spider):
        print()
        return item
```

Раннер `runner.py` настроенный для одного паука 
```
if __name__ == '__main__':
    configure_logging() 
    
# settings импортируется посредствам вызова функции. 
# когда мы вызываем функцию get_project_settings()
# эта функция настроена на поиск в нашем виртуальном окружении 
# файла settings.py и импорт его внутрь нашего проекта, внутрь паука
    
    settings = get_project_settings()
    runner = CrawlerRunner(settings)
    
    runner.crawl(AvitoSpider, query='bmw')

    reactor.run()
```


Стандартный набор с которого мы начинаем нашу работу. 
Работать будем с сайтом Avito.ru. Наша задача автоматизировать сбор данных плюс сверху скачать фотографии ещё. Причём скачать так, чтобы скаченные данные были связаны с фотографиями. Чтобы получая информацию об одном объекте скаченном получали информацию и фотки. 

Начинаем всегда с того, что переходим на сайт, в нашем случае avito - вводим что ищем (например iphone), открывается страница с результатами поиска - это и будет наша точка входа. вставляем эту точка входа в файл `spiders/avito.py`. 

In [ ]:
start_urls = [f'https://www.avito.ru/moscow?q=iphone']

Попробуем улучшить. `?q=iphone` - параметр зашит в ссылку и получается приложение настроено только на поиск iphone. А нам хочется передавать параметр поиска через какую нибудь переменную, при чтении из окна интерфейса или из стороннего файла. То есть влиять на работу приложения посредствам ввода запроса снаружи. Поэтому изменим начальную структуру нашего паука. 

Нам необходимо будет понять как внутрь класса `AvitoSpider` можем передать какие нибудь параметры. При создании объекта класса как передавать внутрь параметры? Что должно быть в самом классе, чтобы можно было снаружи параметры принимать и сохронять в свойствах объекта. НУЖЕН КОНСТРУКТОР. Здесь у spiders/avito.py конструктора нет, 
но мы наследуемся от класса `scrapy.Spider` можно в ИДЕ с зажатой клавишей CTRL перейти в этот класс (навелись мышкой на `scrapy.Spider` и с контролом переходим в код с классом.) и видим что в родительском классе конструктор всё таки есть. `def __init__(self, name=None, **kwargs)`. 

Внутри нашего класса `AvitoSpider(scrapy.Spider)` можем тоже определить этот самый конструктор `def __init__(self, )` и здесь в конструкторе мы создадим параметры, которые будут приниматься снаружи. Пока сделаем его по умолчанию. Если мы ничего не передадим будем искать то что указано по умолчанию. `def __init__(self, query = 'iphone'):`. Можно сделать и kwargs `def __init__(self, **kwargs):`, чтобы можно было передавать любой параметр. Если мы наследуемся от родительского класса и у него есть конструктор, мы его должны переопределять в дочернем классе. Поэтому внутрь класса обязательно `super().__init__(**kwargs)`

```
def __init__(self, **kwargs):
    super().__init__(**kwargs)
```

Теперь логика не нарушена, инициализация какой была такой и осталась. Внутри самого конструктора добавляем свой собственный функционал, ради которого мы и решили переопределить конструктор. В чвстности нам нужно через kwargs получить значение поиска который передаётся в `start_urls = [f'https://www.avito.ru/moscow?q=iphone']`. Поэтому мы `start_urls =` из атрибутов класса превратим в свойства будущих объектов. Мы убираем его из `class AvitoSpider` и вставляем внутрь конструктора через self. а также делаем f строку и вставляем kwargs `self.start_urls = [f'https://www.avito.ru/moscow?q={kwargs.get('query')}']` В параметре query будет значение для строки поиска. 

Как мы в чужом фреймворке, в чужом коде где мы сможем передать параметр query внутрь класса `AvitoSpider(scrapy.Spider)`. query стал обязательным параметром и мы теперь не сможем создать объект класса `AvitoSpider` без передачи параметра query внутрь. 

Все параметры которые лягут в kwargs мы указываем при инициализации объекта методом crawl в runner.py. `runner.crawl(AvitoSpider, query='bmw')`. 

Теперь параметр query получает значение и в момент создания объекта класса паука в конструкторе мы инициализируем свойство `start_urls =` который принимает в себя этот самый параметр query. 

In [ ]:
# spiders/avito.py

import scrapy
from scrapy.http import HtmlResponse
from avitoparser.items import AvitoparserItem
from scrapy.loader import ItemLoader

class AvitoSpider(scrapy.Spider):
    name = 'avito'
    allowed_domains = ['avito.ru']

    def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.start_urls = [f'https://www.avito.ru/moscow?q={kwargs.get('query')}']

Запустили через раннер в режиме отладки. Посмотрели как отрабатывает паук при ручном запуске через ранер. Дальше нужно писать логику сбора данных в файле `spiders/avito.py` создаём метод `def parse()`
мы проходим по ссылке `https://www.avito.ru/moscow?q={kwargs.get('query')}` и теперь нам нужно собрать ссылки на товары,  которые найдены на странице, чтобы войти внутрь каждого объявления. 

Через инструменты разработчика инспектируем элемент. у нас есть тег а в котором находится ссылка, также у тега находим атрибут 'data-maker'='item-title'. Составляем икс пас и проверяем `//a[@data-marker='item-title']`. 

Теперь в методе parse нашего класса создаём переменную, куда поместим список ссылок, который получим через Xpath. 
`links = response.xpath("//a[@data-marker='item-title']")`. Получили контейнеры теги, хронящие в себе информацию. По хорошему чтобы получить ссылку мы должны писать `links = response.xpath("//a[@data-marker='item-title']/@href")` потом должы написать метод getall() чтобы эти ссылки извлечь и превратить в список данных. `links = response.xpath("//a[@data-marker='item-title']/@href").getall()`. Но в случае с сылками в скрапи есть одна очень интересная настройка. Мы можем передовать в методы для выполнения запросов не конечные финальные ссылки, а объекты которые их содержат. Скрапи умеет сам заходить внутрь и извлекать значение атрибута href.  И вот у нас на руках объекты со ссылками `links = response.xpath("//a[@data-marker='item-title']")` и мы будем итеррироваться по этим объектам так, как будто они уже являются ссылками. 
```
for link in links:
    yield response.follow(link, callback=self.parse_ads)
```
создадим ещё один метод parse_ads `def parse_ads(self, response:HTMLResponse)` для того, чтобы заходить внутрь объявления. В верхнем методе мы используем генератор, для того чтобы возвращались нам результаты и мы шли к следующей ссылке.  

In [ ]:
# spiders/avito.py

    def parse(self, response:HtmlResponse):
        links = response.xpath("//a[@data-marker='item-title']")
        for link in links:
            yield response.follow(link, callback=self.parse_ads)
            
    def parse_ads(self, response:HTMLResponse):
        pass

Мы зашли внутрь объявления. Дальше собираем информацию внутри объявления. Наименование товара. Опять же инспектируем икс пас.
```
<h1 class="style-title-info-title-eHW9V style-title-info-title-text-CoxZd"><span itemprop="name" class="title-info-title-text" data-marker="item-view/title-info">Hyundai Solaris, 2014</span></h1>
``` 
для нимменования товара должны взять текст тега спан, находящегося в теге h1. `name = response.xpath("//h1/span/text()").get()`.

Инспектируем цену
```
<span content="770000" itemprop="price" class="js-item-price style-item-price-text-_w822 text-text-LurtD text-size-xxl-UPhmI">770&nbsp;000</span>
```
Видим что цена содержится в атрибуте тега content="770000". 
дальше берём цену товара `price = response.xpath("//span[@itemprop='price'/@content]").get()`.
Но можно также взять и из текста `price = response.xpath("//span[@itemprop='price'/text()]").get()`,
но эти данные немного грязные. пробел в виде &nbsp; нужно будет дополнительно обрабатывать. 

Добавим также ссылку на объявление `url = response.url`

In [ ]:
    def parse_ads(self, response:HTMLResponse):
        name = response.xpath("//h1/span/text()").get()
        price = response.xpath("//span[@itemprop='price'/@content]").get()
        url = response.url

Следующий элемент, который нам надо собрать это фотографии товара. 
`photos = response.xpath(//)` чтобы сформировать правильный икс пас нам надо понимать как устроены галереии в вёрстке. 
Практически все галереии устроены одинаково. У нас есть мелкие картинки внизу в полном объёме представленные, и только одна большая картинка наверху. Проинспектируем элемент и посмотрим как это всё выглядит в коде. 
```
<div class="gallery-root-n3_HK" data-marker="item-view/gallery"><div class="image-frame-root-vKeXJ"><div class="image-frame-borderWrapper-wCbtU"><div class="image-frame-controlButtonArea-_3TO9 image-frame-controlButton_left-i5XEe" data-delta="-1" data-marker="image-frame/left-button"><button class="image-frame-controlButton-_vPNK"></button></div><div class="image-frame-controlButtonArea-_3TO9 image-frame-controlButton_right-HeBIM" data-delta="1" data-marker="image-frame/right-button"><button class="image-frame-controlButton-_vPNK"></button></div><div class="image-frame-wrapper-_NvbY" data-title="Фитолампа на полный цикл от 150Вт до 300Вт" data-url="https://70.img.avito.st/image/1/1.NqqfYba4mkOpyFhG_xFYqq3DmEUhwBhL6cWYQS_Ikkkp.4G3SgDSvy8V6UgjJPuhP8xh4LRcHYBYibxfs77-DeWI" data-image-id="0" data-marker="image-frame/image-wrapper"><span class="image-frame-cover-lQG1h" style="background-image:url(https://70.img.avito.st/image/1/1.NqqfYba4mkOpyFhG_xFYqq3DmEUhwBhL6cWYQS_Ikkkp.4G3SgDSvy8V6UgjJPuhP8xh4LRcHYBYibxfs77-DeWI)"></span><img src="https://70.img.avito.st/image/1/1.NqqfYba4mkOpyFhG_xFYqq3DmEUhwBhL6cWYQS_Ikkkp.4G3SgDSvy8V6UgjJPuhP8xh4LRcHYBYibxfs77-DeWI" alt="Фитолампа на полный цикл от 150Вт до 300Вт" class="desktop-1ky5g7j"></div><div class="image-frame-hidden-XSrK5"><button class="videoPlayer-button-CQ5Qh buttonSizeS"></button><div class="videoPlayer-hidden-UR5_1"></div></div></div></div><ul class="images-preview-previewWrapper-R_a4U images-preview-previewWrapper_newStyle-fGdrG" data-marker="image-preview/preview-wrapper"><li class="images-preview-previewImageWrapper-RfThd images-preview-previewImageWrapper_selected-OgdIL images-preview-previewImageWrapperWithPointer-NioZh" data-marker="image-preview/item" data-index="0" data-type="image"><img src="https://70.img.avito.st/image/1/1.NqqfYba3mkO_w_aX-gJzxSjCmkkhVJkvKcI.P6Kcq8tPOZ6ca1qz2TA7N-_EZwHWaRMeh4DtwAZG88A" alt="Фитолампа на полный цикл от 150Вт до 300Вт" srcset="" class="desktop-1i6k59z"></li><li class="images-preview-previewImageWrapper-RfThd images-preview-previewImageWrapperWithPointer-NioZh" data-marker="image-preview/item" data-index="1" data-type="image"><img src="https://70.img.avito.st/image/1/1.i86u67a3JyeOSUuJzYjOoRlIJy0Q3iRLGEg.VVdj-JZpB0z0SU1YctctOHbUyN2cpjopC1iLQYrSyow" alt="Фитолампа на полный цикл от 150Вт до 300Вт" srcset="" class="desktop-1i6k59z"></li><li class="images-preview-previewImageWrapper-RfThd images-preview-previewImageWrapperWithPointer-NioZh" data-marker="image-preview/item" data-index="2" data-type="image"><img src="https://20.img.avito.st/image/1/1.YKHGM7a3zEjmkaCwolAlznGQzEJ4Bs8kcJA.XKm8ftmBU8pFA4yGZwX1cTG-lQPSMZo9j6ObF7wV5mM" alt="Фитолампа на полный цикл от 150Вт до 300Вт" srcset="" class="desktop-1i6k59z"></li></ul></div>
```
Ссылки на маленькие картинки нам не интересны, оны не несут много информации из-за разрешения картинок. 
Инспектируем одну большую картинку. Идея собрать маленькие картинки под галереей сразу прогорает. 
```
<img src="https://70.img.avito.st/image/1/1.NqqfYba4mkOpyFhG_xFYqq3DmEUhwBhL6cWYQS_Ikkkp.4G3SgDSvy8V6UgjJPuhP8xh4LRcHYBYibxfs77-DeWI" alt="Фитолампа на полный цикл от 150Вт до 300Вт" class="desktop-1ky5g7j">
```
Наша цель не столько скачать картинки изначально, сколько получить ссылки на эти картинки. А скачать дальше это уже читсо технический несложный процесс. 
Выбирем картинку из галереии снизу, она станет полноразмерной и проинспектируем этот элемент.
```
<img src="https://20.img.avito.st/image/1/1.YKHGM7a4zEjwmg5NikIOofSRzk54kk5AsJfOSnaaxEJw.1AtxMPtL-3_KE2ab8mK04mUgxFu25E8OihFSx9T2VQA" alt="Фитолампа на полный цикл от 150Вт до 300Вт" class="desktop-1ky5g7j">
```
идём вверх и смотрим в каком контейнере она лежит. div контейнер class="image-frame-wrapper-_NvbY".

Сейчас картинки подгружаются с помощью джаваскрипт. Поэтому сейчас их собрать становится проблематично. При наведении мышкой на картинку содержение одного и того же тега меняется. Динамическое подгружение. 

На примере маленьких картинок икс пас:`//li[contains(@class,'images-priview')]/img/@src` нам нужно взять все картинки .getall()
получим ссылки, ведущие на маленькие картинки 
`photos = response.xpath("//li[contains(@class,'images-priview')]/img/@src").getall()`


In [ ]:
    def parse_ads(self, response:HTMLResponse):
        name = response.xpath("//h1/span/text()").get()
        price = response.xpath("//span[@itemprop='price'/@content]").get()
        url = response.url
        photos = response.xpath("//li[contains(@class,'images-priview')]/img/@src").getall()

Можно подключить селениум в скрапи и уже с помощью него подгружать динамические данные и дальше уже собирать. Прощёлкать картинки, они прогрузятся и собрать их уже полноразмерные. 

Дальше делаем импорт `from avitoparser.items import AvitoparserItem` и yield для сохранения порциями данных. 

In [ ]:
# spiders/avito.py

import scrapy
from scrapy.http import HtmlResponse
from avitoparser.items import AvitoparserItem

# ...

    def parse_ads(self, response:HTMLResponse):
        name = response.xpath("//h1/span/text()").get()
        price = response.xpath("//span[@itemprop='price'/@content]").get()
        url = response.url
        photos = response.xpath("//li[contains(@class,'images-priview')]/img/@src").getall()
        yield AvitoparcerItem(name=name, price=price, url=url, photos=photos)

И не забываем в items добавить соответствующие поля.

In [ ]:
# items.py

class AvitoparserItem(scrapy.Item):
    # define the fields for your item here like:
    name = scrapy.Field()
    price = scrapy.Field()
    url = scrapy.Field()
    photos = scrapy.Field()

In [ ]:
# pipelines.py

class AvitoparserPipeline:
    def process_item(self, item, spider):
        print()
        return item

Айтомы все скомплектованы, данные все записаны и нам теперь необходимо скачать файлы по представленным в photos ссылкам. В скрапи есть отдельно созданный функционал для этих целей. Этот функционал заложен внутри отдельного pipeline. Пайплайнов несколько для работы с файлами, мы будем пользоваться пайплайном для работы с фотографиями. 

Из соответствующего раздела пайплайнс `from scrapy.pipelines.images import ImagesPipeline`. Теперь создадим класс - ещё один пайплайн. Это нормальный подход, когда есть несколько пайплайнов для обработки наших айтомов.  Удобно с точки зрения структурности, потому что всё разложено по полочкам, функциональность обработки данных не смешивается и в целом для дальнейшей масштабируемости проекта удобно.  создаём класс `class AvitoPhotoPipeline(ImagesPipeline)` и чтобы не придумывать свой собственный функционал будем наследоваться от `ImagesPipeline`. отдельный класс созданный на базе ImagesPipeline даст нам возможность использоовать его методы. 

У нашего пайплайна есть точка входа - это `process_item`. У любого пайплайна должен быть этот самый process_item. Исключения составляют мультимидийные пайплайны, у которых точка входа немного другая. она называется get_media_requests. Вызывается этот метод самым первым и внутри этого метода мы должны написать логику сбора фотографий. ` get_media_requests(self, item, info)` в метод приходит item и ещё создаётся объект info, который содержит информацию о процессе скачивания фотографий: данные о том сколько файлы стоят в осереди на скачивание, сколько сейчас уже скачиваются, сколько осталось ещё скачать. 

сначала проверим есть ли у нашего айтома в поле photos хоть что то.
`if item['photos']:` - если там что то есть и если есть то начинаем скачивать
```
if item['photos']:
    for img in item['photos']:
```

для каждой ссылки мы должны попытаться (try) выполнить запрос по этой ссылке. Если мы делаем без try-except тем более в потоке, так как одновременно можем скачивать несколько фотографий, у нас вероятность ошибки повышается. Поэтому страхуемся, чтобы приложение не упало. Внутри try мы как раз производим скачивание.
```
if item['photos']:
    for img in item['photos']:
        try:
            yield 
```

Процедура скачивания. мы создаём новый объект запроса. Первичный объект запроса у нас уже создавался `def parse(self, response:HtmlResponse)` - объект response. Дальше мы работаем в рамках этого объекта. при переходе на следующую ссылку опять делаем через response. Условно говоря это наша сессия в браузере - наша открытая вкладка. 

А процесс качивания фотографий это новая открытая вкладка у нас в браузере.  Технически каждый процесс скачивания - это отдельный процесс, который не зависит от основоного.  Здесь это выражено в качестве создания нового объекта запроса, поэтому ещё к нашему пайплайну подключаем объект скрапи так как именно в модуле скрапи у нас находится объект Request. `import scrapy ` внутрь класса Request(img) передаём ссылку для скачивания картинки. Просто создаём по отдельному объекту запроса для каждой ссылки ведущей нас на фотографию. 

Дальше делаем except. Выведим сообщение об ошибке. 

In [ ]:
import scrapy

class AvitoPhotoPipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        if item['photos']:
            for img in item['photos']:
                try:
                    yield scrapy.Request(img)
                except Exeption as c:
                    print(c)
                    

Для того чтобы всё сработало в полной мере ещё необходимо сделать 3 действия. первое действие касается только лишь картинок. Мы должны установить дополнительно в комплект с модулем скрапи ещё модуль pillow - модуль для обработки фотографий.

In [ ]:
 ! pip install pillow

Второй момент. В settings.py нужно указать параметр `IMAGES_STORE = 'photos'`. Если этого не сделать всё будет хорошо работать, но скачиваться ничего не будет, потому что данное значение будет пустым, а скачивание по значению None не даёт ничего. Физически файл будет скачиваться, но не будет сохроняться на жёсткий диск. 

Третий момент. Мы создали отдельновзятый пайплайн `AvitoPhotoPipeline`. И раз мы это сделали у нас информации об его существовании нет нигде вообще. Информация об AvitoparserPipeline создаётся когда мы генерируем проект, а об AvitoPhotoPipeline информации нету. У нас нет входа в AvitoPhotoPipeline. Это также делается в settings.py 
```
ITEM_PIPELINES = {
   'avitoparser.pipelines.AvitoparserPipeline': 300,
   'avitoparser.pipelines.AvitoPhotosPipeline': 200
}
```

В этом словаре может храниться несколько пайплайнов. Указываем начальный путь до пайплайна `avitoparser.pipelines.AvitoPhotosPipeline` в ключах указываем пути до класса с пайплайнами. Паук должен знать о всех наших пайплайнах. 
Встаёт вопрос в следующем, раз пайплайнов несколько, в каком порядке по ним ходить. 

Цифра, которая является значением ключа задаёт приоритет прохождения. Чем меньше значение, тем более высокий приоритет. Нам надо, чтобы фотки скачиались раньше, чем всё складывается в базу данных. Теперь паук будет точно знать, что item надо сперва закинуть в `AvitoPhotosPipeline` а потом в `AvitoparserPipeline`. 

### Привьюшки
В settings.py можно указать параметр 
```
IMAGES_THUMBS = {
   'medium': (55, 35)
}
```
в словаре мы говорим, что хотим привьюшки другого размера, например medium и указываем размер 55 на 35 пикселей. Для них создастся папка медиум, в которую они будут помещены соответствующего размера. 


### как картинки подогнать под объявления
Теперь нам осталось решить проблему как сопоставить фотографии скаченные и айтомы. Наша задача сделать связи между текстовыми данными  name, price, url и ссылки на фотографии скаченные, которые где то на диске хронятся (в папке full). 

Для этого есть специальный метод, который мы заберём у ImagesPipeline. Метод  `def item_completed(self, results, item, info)` в этом методе есть объект `results`. Это тоже список и каждый объект этого списка это картеж. Для каждого скаченного объекта в списке results создаётся картеж, внутри которого 2 элемента: булиево значение, означающее успешно ли скачалась фотка и второй элемент кортежа - словарь с четеримя полезнейшими ключами. `checksum` - контрольная сумма файла  (чтобы скрапи не скачивал уже скаченный ранее файл).  `path` - удрес и название скаченной фотки, еперь у нас есть путь к скаченной фотке. `status` - говорит о том был уже файл скачен ранее или нет (download - только скачен, updated - скачен был ранее). `url` - ссылка откуда скачен файл. 

Теперь мы это хотим сохранить в свой item. мы возьмём второй элемент кортежа `itm[1]` для каждого `itm` в `results`, но при условии если первый элемент кортежа True (`if itm[0]`)

`item['photos'] = [itm[1] for itm in results if itm[0]]`

Теперь в каждый `item['photos']` запишится этот словарь.  И теперь вместо списка ссылок который раньше был в `item['photos']` записывается более полная информация и мы можем по адресу обратиться к скаченной картинке. 

In [ ]:
class AvitoPhotosPipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        if item['photos']:
            for img in item['photos']:
                try:
                    yield scrapy.Request(img)
                except Exception as e:
                    print(e)

    def item_completed(self, results, item, info):
        item['photos'] = [itm[1] for itm in results if itm[0]]
        return item

После того как завершили работу с items в рамках одного пайплайна мы обязательно должны этот самый item вернуть. `return item`. 

теперь не важно что все собранные фото лежат в одной папке. мы можем обратиться к словарю и получить ссылку на адрес хранения фотографии на жёстком диске. Технически фотографии не хронятся в вамой базе данных, в базе данных хронятся локальные ссылки на эту фотографию

### Ещё о скрапи
У нас есть паук. 
```
# spiders/avito.py

import scrapy
from scrapy.http import HtmlResponse
from avitoparser.items import AvitoparserItem
from scrapy.loader import ItemLoader

class AvitoSpider(scrapy.Spider):
    name = 'avito'
    allowed_domains = ['avito.ru']


    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.start_urls = [f"https://www.avito.ru/izhevsk?q={kwargs.get('query')}"]

    def parse(self, response: HtmlResponse):
        links = response.xpath("//a[@data-marker='item-title']")
        for link in links:
            yield response.follow(link, callback=self.parse_ads)


    def parse_ads(self, response: HtmlResponse):
        loader = ItemLoader(item=AvitoparserItem(), response=response)
        loader.add_xpath('name', '//h1/span/text()')
        loader.add_xpath('price', "//span[@itemprop='price']/text()")
        loader.add_xpath('photos', "//li[contains(@class,'images-preview-preview')]/img/@src")
        loader.add_value('url', response.url)
        yield loader.load_item()


        # name = response.xpath("//h1/span/text()").get()
        # price = response.xpath("//span[@itemprop='price']/text()").get()
        # url = response.url
        # photos = response.xpath("//li[contains(@class,'images-preview-preview')]/img/@src").getall()
        # yield AvitoparserItem(name=name, price=price, url=url, photos=photos)
```

Как сделать так, чтобы паук не занимался извлечением данных вообще. Метод `def parse_ads(self, response: HtmlResponse):`. Процесс itemLoader. Добавим в класс нашего паука ещё один класс. `from scrapy.loader import ItemLoader`. Он нам даёт возможность освободить функциональность паука на конечном методе, который заключается в том, чтобы чисто технически собрать с помощью специальных селекторов данные со страницы.  По сути задача паука - только парсинг. Он не должен заниматься обработкой. Он просто переходит по страницам и собирает данные, а мы эти данные собираем и отправляем в другой модуль. 

В последнем методе, который занимается сбором данных с финальной страницы, полученной на предыдущем этапе ` def parse_ads(self, response: HtmlResponse):`мы должны создать объект этого loader. `loader = ItemLoader()` и внутри в конструктор ему передать 2 параметра: item. по сути здесь мы должны создать объект нашего класса, AvitoparserItem() `loader = ItemLoader(item=AvitoparserItem(), )` вторым моментом мы передаём параметром наш response `loader = ItemLoader(item=AvitoparserItem(), response=response)` - наш объект загрузчика создан. 

Так внутри этого объекта загрузчика мы имеем объект класса AvitoparserItem() то есть структура нашего будущего айтома хронится теперь в loader. Мы теперь должны заполнить эту самую структуру. Мы создавали 4 поля:
```
loader.add_xpath('name', '//h1/span/text()')
        loader.add_xpath('price', "//span[@itemprop='price']/text()")
        loader.add_xpath('photos', "//li[contains(@class,'images-preview-preview')]/img/@src")
        loader.add_value('url', response.url)
```

Теперь мы работаем не напрямую с какими то свойствами, а работаем через loader. У него есть отдельные методы очень похожие на методы response.  add_Xpath - добавить элемент собранный с помощью xpath , add_CSS - добавить элемент собранный с помощью css и add_value - добавить готовое значение.  В готовых методах первым параметром идёт имя поля (ключ), второй параметр значение этого поля - селектор. 

Через loader мы заполнили все наши поля. И в финале когда мы уже знаем все селекторы применённые для всех полей делаем через yield вызов метода load_item() `yield loader.load_item()`. 

Мы убрали у паука функциональность по обработке данных. Мы на этом этапе передаём всю обработку и извлечение данных уже в другой модуль. В обёртку над нашим классом AvitoparserItem().
```
class AvitoparserItem(scrapy.Item):
    # define the fields for your item here like:
    name = scrapy.Field(output_processor=TakeFirst())
    price = scrapy.Field(input_processor=MapCompose(clear_price), output_processor=TakeFirst())
    url = scrapy.Field(output_processor=TakeFirst())
    photos = scrapy.Field()
    _id = scrapy.Field()

```

Тем самым паук может заниматься своими делами дальше, а ItemLoader будет заниматься уже первичной обработкой данных и их структурированием. Мы тем самым распараллеливаем процессы и можем выигрывать в скорости. 

Польза также в том, что мы можем делать мелкие обработки до пайплайн. ItemLoader забрал часть паука и часть пайплайнов. Мы можем здесь добавить обработчики. у него есть свои собственные. `from itemLoaders.processors import MapCompose, TakeFirst` мы можем импортировать 6 обработчиков из которых реально используются только 2-3. 

обработчики нужны, чтобы собранные данные когда они попадут в соответствующие поля их можно было по мелкому ка книбудь обработать. В частности не забываем что методы xpath и css селекторы всегда нам возвращают список. И нам нужно этот список раскрыть, достав оттуда первый элемент, когда нам это необходимо. Для этого используем обработчик `TakeFirst`. 

Все обработчики делятся на постобработчики и предобработчики. Они жёстко зафиксированы и изменять их нельзя. Каждое поле позволяет использовать один постобработчик и один предобработчик. `TakeFirst` - постобработчик. Этот постобработчик мы передаём в конструкторе класса AvitoparserItem в соответствующее поле в значении параметра output_processor= `name = scrapy.Field(output_processor=TakeFirst())`. Когда сработает этот обработчик он извлечёт из полученного списка первое значение и его сохранит в поле name. 

Для обработки цены нужен `MapCompose`. У нас приходит цена с пробелами, которые заменены спецсимволами и нам надо эти спецсимволы убрать. И эту мелкую предобработку можно производить внутри ItemLoader. Мы используем предобработчик `input_processor=MapCompose`. `price = scrapy.Field(input_processor=MapCompose(clear_price), output_processor=TakeFirst())`. Суть - предобработчик берёт список, который находится внутри данного поля и к каждому элементу списка применяет уазанную в параметрах функцию. 

Создадим функцию обработки `def clear_price()` Чтобы эта функция могла что то обрабатывать, она должна уметь что то принять. `def clear_price(value):`

In [ ]:
def clear_price(value):
    if value:
        value = value.replace('\xa0','')
        try:
            value = int(value)
        except:
            return value
        return value

`price = scrapy.Field(input_processor=MapCompose(clear_price), output_processor=TakeFirst())` так как нам возвращается список из 2ух элементов, то возьмём лишь одну цену с помощью постобработчика `output_processor=TakeFirst()`. 

input_processor всегда срабатывает раньше output_processor

# Файлы с архива урока 

In [ ]:
# spiders/avito.py

import scrapy
from scrapy.http import HtmlResponse
from avitoparser.items import AvitoparserItem
from scrapy.loader import ItemLoader

class AvitoSpider(scrapy.Spider):
    name = 'avito'
    allowed_domains = ['avito.ru']


    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.start_urls = [f"https://www.avito.ru/izhevsk?q={kwargs.get('query')}"]

    def parse(self, response: HtmlResponse):
        links = response.xpath("//a[@data-marker='item-title']")
        for link in links:
            yield response.follow(link, callback=self.parse_ads)


    def parse_ads(self, response: HtmlResponse):
        loader = ItemLoader(item=AvitoparserItem(), response=response)
        loader.add_xpath('name', '//h1/span/text()')
        loader.add_xpath('price', "//span[@itemprop='price']/text()")
        loader.add_xpath('photos', "//li[contains(@class,'images-preview-preview')]/img/@src")
        loader.add_value('url', response.url)
        yield loader.load_item()


        # name = response.xpath("//h1/span/text()").get()
        # price = response.xpath("//span[@itemprop='price']/text()").get()
        # url = response.url
        # photos = response.xpath("//li[contains(@class,'images-preview-preview')]/img/@src").getall()
        # yield AvitoparserItem(name=name, price=price, url=url, photos=photos)

In [ ]:
# settings.py

# Scrapy settings for avitoparser project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'avitoparser'

IMAGES_STORE = 'photos'
IMAGES_THUMBS = {
   'medium': (55, 35)
}


SPIDER_MODULES = ['avitoparser.spiders']
NEWSPIDER_MODULE = 'avitoparser.spiders'

# Crawl responsibly by identifying yourself (and your website) on the user-agent
USER_AGENT = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
CONCURRENT_REQUESTS = 8

LOG_ENABLED = True
LOG_LEVEL = "DEBUG"

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
DOWNLOAD_DELAY = 1.5
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
COOKIES_ENABLED = True

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
# DEFAULT_REQUEST_HEADERS = {
#   'cookie':'yandexuid=1259160151646070856; yuidss=1259160151646070856; yabs-sid=1937487681646070856; ymex=1961430856.yrts.1646070856#1961430856.yrtsi.1646070856; gdpr=0; _ym_uid=1646070893156269093; my=YwA=; amcuid=8340317491646422388; yandex_login=Orleon.ya; i=wdTJiryHb+LLnT5mcRSlLhkbNWoRkyWlPsQzRYQZNAx4MhNo6TM/BV8R51m0d0jIkpBJv0xmNtTmxeXkiTiWht9wDgw=; is_gdpr=0; is_gdpr_b=CNaZZBDuaigC; yandex_gid=44; cycada=Pu5H3ElFTWHQAAU7lrMwAL/hvOtk0vJNciYyzRBtN1E=; _ym_d=1649519883; yabs-frequency=/5/0000000000000000/LVTwO9j8Vb84HY6q2tDmb00000H68M-bN1mQii9h14Od8krF10oancu4HYC0/; yp=1665287886.szm.1_25:2560x1440:1990x1072#1962472582.udn.cDpPcmxlb24ueWE%3D#1651771282.ygu.1#1651771283.spcs.l#1651857692.csc.1; ys=udn.cDpPcmxlb24ueWE%3D#c_chck.8048816; Session_id=3:1649696374.5.0.1647112582511:GiqSWw:25.1.2:1|271833109.0.2|3:250771.178521.H07E8OxfqD1kCKQ9zmuC7qex_qY; sessionid2=3:1649696374.5.0.1647112582511:GiqSWw:25.1.2:1|271833109.0.2|3:250771.178521.H07E8OxfqD1kCKQ9zmuC7qex_qY'
# }

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'avitoparser.middlewares.AvitoparserSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'avitoparser.middlewares.AvitoparserDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'avitoparser.pipelines.AvitoparserPipeline': 300,
   'avitoparser.pipelines.AvitoPhotosPipeline': 200
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'


In [ ]:
# runner.py 

from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
from scrapy.utils.project import get_project_settings

from avitoparser.spiders.avito import AvitoSpider

if __name__ == '__main__':
    configure_logging()
    settings = get_project_settings()
    runner = CrawlerRunner(settings)
    # query = input('')
    runner.crawl(AvitoSpider, query='bmw')

    reactor.run()

In [ ]:
# test.py

import requests
import wget

url = 'https://www.ap22.ru/netcat_files/multifile/2546/90766/palms_3242342_960_720.jpg'
wget.download(url)

# response = requests.get(url)
#
# with open('palms.jpg', 'wb') as f:
#     f.write(response.content)
#
# response = requests.get(url, stream=True)
# handle = open(target_path, "wb")
# for chunk in response.iter_content(chunk_size=512):
#     if chunk:  # filter out keep-alive new chunks
#         handle.write(chunk)

In [ ]:
# pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
import scrapy
from itemadapter import ItemAdapter
from scrapy.pipelines.images import ImagesPipeline

class AvitoparserPipeline:
    def process_item(self, item, spider):
        print()
        return item


class AvitoPhotosPipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        if item['photos']:
            for img in item['photos']:
                try:
                    yield scrapy.Request(img)
                except Exception as e:
                    print(e)

    def item_completed(self, results, item, info):
        item['photos'] = [itm[1] for itm in results if itm[0]]
        return item

In [ ]:
# items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy
from itemloaders.processors import MapCompose, TakeFirst

def clear_price(value):
    if value:
        value = value.replace('\xa0','')
        try:
            value = int(value)
        except:
            return value
        return value



class AvitoparserItem(scrapy.Item):
    # define the fields for your item here like:
    name = scrapy.Field(output_processor=TakeFirst())
    price = scrapy.Field(input_processor=MapCompose(clear_price), output_processor=TakeFirst())
    url = scrapy.Field(output_processor=TakeFirst())
    photos = scrapy.Field()
    _id = scrapy.Field()

